In [1]:
import torch
from torch import nn
import os
import cv2
import matplotlib.pyplot as plt
import  numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, models
# import gdown

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

# face_localization_model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

cuda:0


Upload your kaggle API key to download dataset

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

!kaggle datasets download -d stoicstatic/face-recognition-dataset

if not os.path.exists("Extracted Faces"):
    !unzip -q face-recognition-dataset.zip "Extracted Faces/*"

/bin/bash: /home/ubuntu/.venv/bin/kaggle: /home/ubuntu/tihan/tihan_eir/venv_tihan/bin/python3: bad interpreter: No such file or directory


### Data preprocessing

In [3]:
data = torch.load("face_data.pth")

train_dataset = data["train"]
test_dataset = data["test"]

In [12]:
batch_size = 8
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### Model

In [44]:
# model creation
additional_layer = nn.Linear(2048, 256)

resnet = models.resnet50(weights="DEFAULT")
encoder_layers = list(resnet.children())[:-1]
encoder_model = torch.nn.Sequential(*encoder_layers)

for param in encoder_model.parameters():
    param.requires_grad = False

model = nn.Sequential(encoder_model, nn.Flatten(), nn.Linear(2048, 256)).to(device)

In [48]:
def train(model, lr, n_epochs, dataloader):
    optimizer = torch.optim.SGD(model.parameters(), lr, weight_decay=0.0001, momentum=0.9)

    for i in range(n_epochs):
        epoch_loss = 0
        print(accuracy(model, dataloader)[0], accuracy(model, test_dataloader)[0])
        
        for input, label in dataloader:
            input = input.to(device)
            label = label.to(device)
            combined_input = torch.cat((input[:, 0:3], input[:, 3:]), dim=0)
            output = model(combined_input)
            val1, val2 = torch.split(output, output.shape[0] // 2)

            distance = nn.functional.pairwise_distance(val1, val2, keepdim=True)
            # print(distance)
            margin = 0.5
            loss = torch.mean((label) * torch.pow(distance, 2) + (1 - label) * torch.pow(torch.clamp(margin - distance, min=0.0), 2))

            epoch_loss += loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("loss: ", epoch_loss.item() / len(dataloader))

    print(accuracy(model, dataloader)[0], accuracy(model, test_dataloader)[0])

# model = SiameseCNN().to(device)

def accuracy(model, dataloader):
    model.eval()
    confusion_matrix = np.zeros((2, 2))


    for input, label in dataloader:
        input = input.to(device)
        label = label.to(device)
        combined_input = torch.cat((input[:, 0:3], input[:, 3:]), dim=0)
        output = model(combined_input)
        val1, val2 = torch.split(output, output.shape[0] // 2)

        distance = nn.functional.pairwise_distance(val1, val2, keepdim=True)
        margin = 0.5
        pred = (distance < margin).int()

        confusion_matrix[0][0] += torch.sum((pred == 1) & (label == 1)).item()
        confusion_matrix[0][1] += torch.sum((pred == 1) & (label == 0)).item()
        confusion_matrix[1][0] += torch.sum((pred == 0) & (label == 1)).item()
        confusion_matrix[1][1] += torch.sum((pred == 0) & (label == 0)).item()

        correct_ratio = np.trace(confusion_matrix) / np.sum(confusion_matrix)

    return correct_ratio, confusion_matrix / np.sum(confusion_matrix)

In [49]:
train(model, .01, 20, train_dataloader)
torch.save(model, "face_recognition_model.pth")

0.5005 0.49925
loss:  0.1222837890625
0.5098 0.51525
loss:  0.0886573974609375
0.507675 0.507
loss:  0.07769096069335937
0.502125 0.504
loss:  0.07721958618164063
0.500775 0.5015
loss:  0.07810003662109374


KeyboardInterrupt: 

In [ ]:
# model = torch.load("face_recognition_model3.pth", map_location=device)
# model.eval()

print(accuracy(model, test_dataloader))
# print(accuracy(model, train_dataloader))

(0.812, array([[0.391, 0.079],
       [0.109, 0.421]]))


In [ ]:
def testImgPreprocess(img_path):
    img = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)
    cordinates = face_localization_model.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)
    print(cordinates)
    x, y, w, h = cordinates[0]
    img = img[x:x+w, y:y+h]

    img = img / 255
    img = cv2.resize(img, (128, 128))

    plt.imshow(img)
    plt.show()

    return img

img1 = testImgPreprocess("test1.jpg")
img2 = testImgPreprocess("test2.jpg")
img3 = testImgPreprocess("test3.jpg")

accuracy([[img2, img3]], [0])

dummy_input = torch.rand((1, 1, 128, 128)).to(device)
torch.onnx.export(model, dummy_input, "face_detection.onnx")

[ WARN:0@1089.472] global loadsave.cpp:248 findDecoder imread_('test1.jpg'): can't open/read file: check file path/integrity


NameError: name 'face_localization_model' is not defined